In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [4]:
import os
if not os.path.isdir('saved_models_{}'.format('gan')):
    os.mkdir('saved_models_{}'.format('gan'))
f = open('saved_models_{}/log_collapse1.txt'.format('gan'), mode='w')
import cv2
import torch.utils.data as Data

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam #optimizer of keras

import matplotlib.pyplot as plt

import sys

import numpy as np
import os
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels) #shape of image
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5) #optimizer of gan

        # Build and compile the discriminator,only to keras
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))  #Input():用来实例化一个keras张量
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, y_test) = fashion_mnist.load_data()

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        X_test = np.expand_dims(X_test, axis=3)
        # y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Train the generator (to have the discriminator label samples as valid)
                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (
                epoch, global_step, d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=40, batch_size=64, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

epoch:0 step:80 [D loss: 0.043489, acc.: 99.22%] [G loss: 3.985941]
epoch:0 step:81 [D loss: 0.053799, acc.: 100.00%] [G loss: 4.132371]
epoch:0 step:82 [D loss: 0.037805, acc.: 100.00%] [G loss: 4.003183]
epoch:0 step:83 [D loss: 0.036968, acc.: 100.00%] [G loss: 3.879628]
epoch:0 step:84 [D loss: 0.044919, acc.: 100.00%] [G loss: 3.923260]
epoch:0 step:85 [D loss: 0.061386, acc.: 100.00%] [G loss: 4.237591]
epoch:0 step:86 [D loss: 0.073392, acc.: 99.22%] [G loss: 4.005834]
epoch:0 step:87 [D loss: 0.075656, acc.: 98.44%] [G loss: 4.308805]
epoch:0 step:88 [D loss: 0.077506, acc.: 98.44%] [G loss: 3.787255]
epoch:0 step:89 [D loss: 0.070413, acc.: 97.66%] [G loss: 4.239999]
epoch:0 step:90 [D loss: 0.045084, acc.: 100.00%] [G loss: 4.442176]
epoch:0 step:91 [D loss: 0.060369, acc.: 100.00%] [G loss: 4.199509]
epoch:0 step:92 [D loss: 0.053730, acc.: 100.00%] [G loss: 4.260989]
epoch:0 step:93 [D loss: 0.081690, acc.: 98.44%] [G loss: 4.227237]
epoch:0 step:94 [D loss: 0.099869, acc.:

Instructions for updating:
Use tf.gfile.GFile.
##############
[ 4.35050724  6.87274919  4.48101285  7.2320855   3.37031399 10.27426719
  6.31613501  7.504254    4.84565664  8.14868929]
##########
epoch:0 step:201 [D loss: 0.478117, acc.: 78.91%] [G loss: 1.955027]
epoch:0 step:202 [D loss: 0.533448, acc.: 74.22%] [G loss: 2.155856]
epoch:0 step:203 [D loss: 0.583235, acc.: 74.22%] [G loss: 2.167778]
epoch:0 step:204 [D loss: 0.788427, acc.: 60.16%] [G loss: 1.944820]
epoch:0 step:205 [D loss: 0.542409, acc.: 74.22%] [G loss: 2.066016]
epoch:0 step:206 [D loss: 0.516660, acc.: 78.12%] [G loss: 1.934346]
epoch:0 step:207 [D loss: 0.472558, acc.: 77.34%] [G loss: 2.116593]
epoch:0 step:208 [D loss: 0.601694, acc.: 69.53%] [G loss: 1.872287]
epoch:0 step:209 [D loss: 0.489880, acc.: 79.69%] [G loss: 1.946437]
epoch:0 step:210 [D loss: 0.590220, acc.: 74.22%] [G loss: 2.025565]
epoch:0 step:211 [D loss: 0.790653, acc.: 50.78%] [G loss: 1.813059]
epoch:0 step:212 [D loss: 0.636002, acc.: 65.

epoch:0 step:316 [D loss: 0.577951, acc.: 73.44%] [G loss: 1.020572]
epoch:0 step:317 [D loss: 0.686783, acc.: 57.03%] [G loss: 0.984415]
epoch:0 step:318 [D loss: 0.669792, acc.: 59.38%] [G loss: 0.985197]
epoch:0 step:319 [D loss: 0.651394, acc.: 64.84%] [G loss: 0.946735]
epoch:0 step:320 [D loss: 0.658933, acc.: 56.25%] [G loss: 0.901718]
epoch:0 step:321 [D loss: 0.647998, acc.: 61.72%] [G loss: 0.930086]
epoch:0 step:322 [D loss: 0.622871, acc.: 64.84%] [G loss: 0.920117]
epoch:0 step:323 [D loss: 0.674205, acc.: 54.69%] [G loss: 0.882902]
epoch:0 step:324 [D loss: 0.659460, acc.: 61.72%] [G loss: 0.877337]
epoch:0 step:325 [D loss: 0.584786, acc.: 65.62%] [G loss: 0.904281]
epoch:0 step:326 [D loss: 0.671191, acc.: 56.25%] [G loss: 0.946370]
epoch:0 step:327 [D loss: 0.609741, acc.: 71.09%] [G loss: 0.915879]
epoch:0 step:328 [D loss: 0.654802, acc.: 60.16%] [G loss: 0.920569]
epoch:0 step:329 [D loss: 0.644707, acc.: 58.59%] [G loss: 0.909157]
epoch:0 step:330 [D loss: 0.642364

epoch:0 step:435 [D loss: 0.589563, acc.: 69.53%] [G loss: 0.878412]
epoch:0 step:436 [D loss: 0.622669, acc.: 70.31%] [G loss: 0.874695]
epoch:0 step:437 [D loss: 0.587486, acc.: 71.09%] [G loss: 0.906179]
epoch:0 step:438 [D loss: 0.585871, acc.: 67.97%] [G loss: 0.921165]
epoch:0 step:439 [D loss: 0.650897, acc.: 63.28%] [G loss: 0.908506]
epoch:0 step:440 [D loss: 0.690593, acc.: 57.03%] [G loss: 0.856414]
epoch:0 step:441 [D loss: 0.643830, acc.: 60.16%] [G loss: 0.917351]
epoch:0 step:442 [D loss: 0.643601, acc.: 66.41%] [G loss: 0.863472]
epoch:0 step:443 [D loss: 0.650120, acc.: 62.50%] [G loss: 0.862136]
epoch:0 step:444 [D loss: 0.653897, acc.: 54.69%] [G loss: 0.854973]
epoch:0 step:445 [D loss: 0.606373, acc.: 68.75%] [G loss: 0.900758]
epoch:0 step:446 [D loss: 0.685892, acc.: 57.03%] [G loss: 0.879098]
epoch:0 step:447 [D loss: 0.656681, acc.: 60.16%] [G loss: 0.869827]
epoch:0 step:448 [D loss: 0.636683, acc.: 64.06%] [G loss: 0.821655]
epoch:0 step:449 [D loss: 0.661101

...

epoch:39 step:36989 [D loss: 0.687172, acc.: 54.69%] [G loss: 0.957981]
epoch:39 step:36990 [D loss: 0.652323, acc.: 53.91%] [G loss: 0.977690]
epoch:39 step:36991 [D loss: 0.671863, acc.: 55.47%] [G loss: 0.898180]
epoch:39 step:36992 [D loss: 0.602434, acc.: 68.75%] [G loss: 0.912628]
epoch:39 step:36993 [D loss: 0.644387, acc.: 64.06%] [G loss: 0.930438]
epoch:39 step:36994 [D loss: 0.615314, acc.: 63.28%] [G loss: 0.906877]
epoch:39 step:36995 [D loss: 0.641815, acc.: 60.16%] [G loss: 0.925136]
epoch:39 step:36996 [D loss: 0.643398, acc.: 60.16%] [G loss: 0.992052]
epoch:39 step:36997 [D loss: 0.638198, acc.: 58.59%] [G loss: 0.998317]
epoch:39 step:36998 [D loss: 0.623098, acc.: 64.06%] [G loss: 0.900005]
epoch:39 step:36999 [D loss: 0.623783, acc.: 68.75%] [G loss: 0.929037]
epoch:39 step:37000 [D loss: 0.623221, acc.: 65.62%] [G loss: 0.955602]
##############
[2.94256995 2.82900112 2.0777407  3.99122382 1.42259229 6.75493073
 2.46208209 4.19991288 4.29768082 8.14868929]
########

epoch:39 step:37105 [D loss: 0.666068, acc.: 60.94%] [G loss: 0.926908]
epoch:39 step:37106 [D loss: 0.621938, acc.: 61.72%] [G loss: 0.961439]
epoch:39 step:37107 [D loss: 0.637202, acc.: 67.97%] [G loss: 0.974353]
epoch:39 step:37108 [D loss: 0.644861, acc.: 64.06%] [G loss: 0.953635]
epoch:39 step:37109 [D loss: 0.644310, acc.: 65.62%] [G loss: 0.962527]
epoch:39 step:37110 [D loss: 0.615283, acc.: 63.28%] [G loss: 0.941461]
epoch:39 step:37111 [D loss: 0.681165, acc.: 53.91%] [G loss: 0.958469]
epoch:39 step:37112 [D loss: 0.613415, acc.: 69.53%] [G loss: 1.043622]
epoch:39 step:37113 [D loss: 0.644629, acc.: 61.72%] [G loss: 0.953986]
epoch:39 step:37114 [D loss: 0.626933, acc.: 67.97%] [G loss: 0.898528]
epoch:39 step:37115 [D loss: 0.595458, acc.: 68.75%] [G loss: 0.920897]
epoch:39 step:37116 [D loss: 0.610568, acc.: 64.84%] [G loss: 0.941599]
epoch:39 step:37117 [D loss: 0.658299, acc.: 59.38%] [G loss: 0.969821]
epoch:39 step:37118 [D loss: 0.665670, acc.: 63.28%] [G loss: 0.

epoch:39 step:37221 [D loss: 0.660492, acc.: 61.72%] [G loss: 1.001187]
epoch:39 step:37222 [D loss: 0.612609, acc.: 68.75%] [G loss: 0.956879]
epoch:39 step:37223 [D loss: 0.682508, acc.: 55.47%] [G loss: 0.916668]
epoch:39 step:37224 [D loss: 0.655320, acc.: 62.50%] [G loss: 0.964812]
epoch:39 step:37225 [D loss: 0.638346, acc.: 64.84%] [G loss: 0.960669]
epoch:39 step:37226 [D loss: 0.638457, acc.: 65.62%] [G loss: 0.968227]
epoch:39 step:37227 [D loss: 0.610890, acc.: 64.06%] [G loss: 0.983025]
epoch:39 step:37228 [D loss: 0.582459, acc.: 72.66%] [G loss: 0.990994]
epoch:39 step:37229 [D loss: 0.679898, acc.: 57.81%] [G loss: 0.974003]
epoch:39 step:37230 [D loss: 0.586259, acc.: 70.31%] [G loss: 0.869319]
epoch:39 step:37231 [D loss: 0.637616, acc.: 64.06%] [G loss: 0.953828]
epoch:39 step:37232 [D loss: 0.633906, acc.: 61.72%] [G loss: 0.975811]
epoch:39 step:37233 [D loss: 0.653064, acc.: 59.38%] [G loss: 0.973956]
epoch:39 step:37234 [D loss: 0.660903, acc.: 57.03%] [G loss: 0.

epoch:39 step:37338 [D loss: 0.616716, acc.: 64.06%] [G loss: 1.016601]
epoch:39 step:37339 [D loss: 0.666609, acc.: 63.28%] [G loss: 1.000219]
epoch:39 step:37340 [D loss: 0.636404, acc.: 65.62%] [G loss: 0.967287]
epoch:39 step:37341 [D loss: 0.657200, acc.: 55.47%] [G loss: 0.926739]
epoch:39 step:37342 [D loss: 0.643556, acc.: 63.28%] [G loss: 0.981456]
epoch:39 step:37343 [D loss: 0.614539, acc.: 65.62%] [G loss: 0.967320]
epoch:39 step:37344 [D loss: 0.635620, acc.: 63.28%] [G loss: 0.955403]
epoch:39 step:37345 [D loss: 0.700367, acc.: 53.91%] [G loss: 0.941830]
epoch:39 step:37346 [D loss: 0.642681, acc.: 61.72%] [G loss: 0.999905]
epoch:39 step:37347 [D loss: 0.651425, acc.: 60.16%] [G loss: 0.929522]
epoch:39 step:37348 [D loss: 0.641344, acc.: 61.72%] [G loss: 0.904368]
epoch:39 step:37349 [D loss: 0.627622, acc.: 62.50%] [G loss: 0.926546]
epoch:39 step:37350 [D loss: 0.661879, acc.: 53.91%] [G loss: 0.981220]
epoch:39 step:37351 [D loss: 0.628412, acc.: 66.41%] [G loss: 1.

epoch:39 step:37456 [D loss: 0.625457, acc.: 62.50%] [G loss: 0.893860]
epoch:39 step:37457 [D loss: 0.627559, acc.: 63.28%] [G loss: 0.981777]
epoch:39 step:37458 [D loss: 0.643228, acc.: 59.38%] [G loss: 1.024500]
epoch:39 step:37459 [D loss: 0.672864, acc.: 57.03%] [G loss: 0.926458]
epoch:39 step:37460 [D loss: 0.632415, acc.: 62.50%] [G loss: 0.903006]
epoch:39 step:37461 [D loss: 0.693866, acc.: 58.59%] [G loss: 0.865650]
epoch:39 step:37462 [D loss: 0.648706, acc.: 60.94%] [G loss: 0.983708]
epoch:39 step:37463 [D loss: 0.587675, acc.: 67.97%] [G loss: 1.000249]
epoch:39 step:37464 [D loss: 0.646008, acc.: 61.72%] [G loss: 0.939110]
epoch:39 step:37465 [D loss: 0.667738, acc.: 59.38%] [G loss: 0.925648]
epoch:39 step:37466 [D loss: 0.679799, acc.: 54.69%] [G loss: 0.968219]
epoch:39 step:37467 [D loss: 0.640472, acc.: 57.81%] [G loss: 0.940898]
epoch:39 step:37468 [D loss: 0.668066, acc.: 58.59%] [G loss: 0.913846]
epoch:39 step:37469 [D loss: 0.609338, acc.: 64.84%] [G loss: 0.